In [1]:
!pip install pyspark
!pip install pyyaml
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import col, row_number


Note: you may need to restart the kernel to use updated packages.


In [2]:
spark = SparkSession.builder.appName("USVehicleAccidentAnalysis").getOrCreate()

In [8]:
Charges_df = spark.read.option("inferSchema", "true").csv('/content/Charges_use.csv', header=True)
Damages_df = spark.read.option("inferSchema", "true").csv('/content/Damages_use.csv', header=True)
Endorse_df = spark.read.option("inferSchema", "true").csv('/content/Endorse_use.csv', header=True)
Primary_Person_df = spark.read.option("inferSchema", "true").csv('/content/Primary_Person_use.csv', header=True)
Restrict_df = spark.read.option("inferSchema", "true").csv('/content/Restrict_use.csv', header=True)
Units_df = spark.read.option("inferSchema", "true").csv('/content/Units_use.csv', header=True)

AnalysisException: Path does not exist: file:/Data/Damages_use.csv

In [ ]:
Charges_df.show(5)

+--------+--------+--------+--------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|              CHARGE|CITATION_NBR|
+--------+--------+--------+--------------------+------------+
|14768622|       1|       1|DRIVING WHILE INT...|        null|
|14838637|       1|       1|                 DWI|  1600000015|
|14838641|       1|       1|RAN RED LIGHT SOL...|      L20440|
|14838641|       2|       1|NO DRIVER'S LICEN...|      L23141|
|14838668|       1|       1|DRIVING WHILE INT...|TX4IC50SRJD3|
+--------+--------+--------+--------------------+------------+
only showing top 5 rows



In [ ]:
Damages_df.show(5)

+--------+----------------+
|CRASH_ID|DAMAGED_PROPERTY|
+--------+----------------+
|14768622|         MAILBOX|
|14768622|     YARD, GRASS|
|14838668|       GUARDRAIL|
|14838685|       ROAD SIGN|
|14838693|    2009 MAZDA 3|
+--------+----------------+
only showing top 5 rows



In [ ]:
Endorse_df.show(5)

+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
+--------+--------+------------------+
only showing top 5 rows



In [ ]:
Primary_Person_df.show(5)

+--------+--------+--------+------------+------------------+--------------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+--------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|   PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|       PRSN_REST_ID|   PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|    DRVR_LIC_TYPE_ID|DRVR_LIC_STATE_ID|   DRVR_LIC_CLS_ID|DRVR_ZIP

In [ ]:
Restrict_df.show(5)

+--------+--------+--------------------+
|CRASH_ID|UNIT_NBR| DRVR_LIC_RESTRIC_ID|
+--------+--------+--------------------+
|14768622|       1|                NONE|
|14838637|       1|  OTHER/OUT OF STATE|
|14838637|       2|                NONE|
|14838641|       1|WITH CORRECTIVE L...|
|14838641|       2|          UNLICENSED|
+--------+--------+--------------------+
only showing top 5 rows



In [ ]:
Units_df.show(5)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+------------+----------+--------------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+---------------+--------------------+--------------------+------------------+-------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID| VEH_MAKE_ID|VEH_MOD_ID|    VEH_BODY_STYL_ID|EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|  VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|VEH_TRANSP_NAME|     VEH_TRANSP_DEST|  CONTRIB_FACT

In [ ]:
Primary_Person_df.select("PRSN_INJRY_SEV_ID").distinct().show(truncate = False)

In [ ]:
Primary_Person_df.select("PRSN_GNDR_ID").distinct().show(truncate = False)

In [ ]:
df2 = spark.createDataFrame([Row()])
type(df2) == type(df1)

True

In [ ]:
#Find the number of crashes (accidents) in which number of persons killed are male?
ans = Primary_Person_df.filter((Primary_Person_df.PRSN_INJRY_SEV_ID=="KILLED") & (Primary_Person_df.PRSN_GNDR_ID == "MALE")).count()
print(type(ans))
df1 = spark.createDataFrame([Row(TOTAL_MALE_CASUALITIES=ans)])
print(type(df2))
df1.write.format('com.databricks.spark.csv').save("../myfile.csv",header = 'true')

<class 'int'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
import csv
data = [[1, 'Number of crashes', df]]
with open('output.csv','w') as out:
    for row in data:
        for col in row:
            out.write('{0},'.format(col))
        out.write('\n')

In [ ]:
Units_df.select("VEH_BODY_STYL_ID").distinct().show(truncate = False)

+---------------------------------+
|VEH_BODY_STYL_ID                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|POLICE MOTORCYCLE                |
|NOT REPORTED                     |
+---------------------------------+



In [ ]:
#How many two wheelers are booked for crashes
Units_df.select("CRASH_ID").filter(Units_df.VEH_BODY_STYL_ID.contains("MOTORCYCLE")).distinct().count()


In [ ]:
Units_df.select("")

In [ ]:
#Which state has highest number of accidents in which females are involved
df = Primary_Person_df.select("*").filter(col("PRSN_GNDR_ID") == "FEMALE").groupby("DRVR_LIC_STATE_ID").count().orderBy(col("count").desc()).show()
df

In [ ]:
Units_df.select("VEH_MAKE_ID").distinct().show(truncate = False)

In [ ]:
resultDf = Primary_Person_df.join(Units_df, on=["CRASH_ID"], how='inner').filter((Units_df.VEH_MAKE_ID != "NA") & \
        ((Primary_Person_df.PRSN_INJRY_SEV_ID.contains("INJURY"))| \
        (Primary_Person_df.PRSN_INJRY_SEV_ID == "KILLED"))).groupby(Units_df.VEH_MAKE_ID).count().withColumnRenamed("count", "total").orderBy("total")
top15DF = resultDf.select("*").orderBy(resultDf.total.desc()).limit(15)
top5to15DF = top15DF.subtract(top15DF.limit(5))
top5to15DF.show()

+------------+-----+
| VEH_MAKE_ID|total|
+------------+-----+
|       HONDA| 4469|
|         GMC| 2065|
|        JEEP| 1735|
|     HYUNDAI| 1725|
|         KIA| 1463|
|    CHRYSLER| 1436|
|FREIGHTLINER| 1219|
|       MAZDA| 1078|
|  VOLKSWAGEN|  947|
|       LEXUS|  931|
+------------+-----+



In [ ]:
Primary_Person_df.select("PRSN_INJRY_SEV_ID").distinct().show(truncate = False)

+-------------------------+
|PRSN_INJRY_SEV_ID        |
+-------------------------+
|NA                       |
|KILLED                   |
|UNKNOWN                  |
|NON-INCAPACITATING INJURY|
|NOT INJURED              |
|POSSIBLE INJURY          |
|INCAPACITATING INJURY    |
+-------------------------+



In [ ]:
#Which are the Top 5th to 15th VEH_MAKE_IDs that contribute to a largest number of injuries including death


In [ ]:
w = Window.partitionBy("VEH_BODY_STYL_ID").orderBy(col("count").desc())
df = Units_df.join(Primary_Person_df, on=['CRASH_ID'], how='inner'). \
            filter(~Units_df.VEH_BODY_STYL_ID.isin(["NA", "UNKNOWN", "NOT REPORTED",
                                                         "OTHER  (EXPLAIN IN NARRATIVE)"])). \
            filter(~Primary_Person_df.PRSN_ETHNICITY_ID.isin(["NA", "UNKNOWN"])). \
            groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").count(). \
            withColumn("row", row_number().over(w)).filter(col("row") == 1).drop("row", "count")

df.write.option("header",True).csv("datacsv")


In [ ]:
Damages_df.select("DAMAGED_PROPERTY").filter(col("DAMAGED_PROPERTY").contains("NO DAMAGE")).show(truncate = False)

+---------------------------------------+
|DAMAGED_PROPERTY                       |
+---------------------------------------+
|NO DAMAGES TO THE CITY POLE 214385     |
|GUARD RAIL (POSSIBLE CONTACT-NO DAMAGE)|
|LIGHT POLE- NO DAMAGE                  |
|NO DAMAGE, SIGNAL LIGHT POLE           |
|CONCRETE BARRIER-NO DAMAGE             |
|NO DAMAGE TO BARRICADE                 |
|CEMENT RETAINING WALL-NO DAMAGE        |
|CONCRETE BARRIER-NO DAMAGE             |
|CONCRETE MEDIAN (NO DAMAGE)            |
|NO DAMAGE TO FENCE                     |
+---------------------------------------+



In [ ]:
joinedDF = Damages_df.join(Units_df, on=["CRASH_ID"], how='inner')

In [ ]:
Damages_df.join(Units_df, on=["CRASH_ID"], how='inner'). \
filter(col("DAMAGED_PROPERTY").contains("NONE")).distinct().count()

192

In [ ]:
noDamagesDF = joinedDf. \
            filter((
                col("DAMAGED_PROPERTY").isin(["NO DAMAGE","NONE"])
            ) | ((
                col("VEH_DMAG_SCL_1_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"])
            ) & (
                col("VEH_DMAG_SCL_2_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"])
            ))).distinct().count()
noDamagesDF

2565

In [ ]:
damageLevelAbove4 = joinedDf. \
            filter((
                (col("VEH_DMAG_SCL_1_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_1_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            ) | (
                (col("VEH_DMAG_SCL_2_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_2_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            )).distinct().count()

damageLevelAbove4

6380

In [ ]:
Units_df.select("FIN_RESP_TYPE_ID").distinct().show(truncate=False)

+----------------------------------------+
|FIN_RESP_TYPE_ID                        |
+----------------------------------------+
|INSURANCE BINDER                        |
|LIABILITY INSURANCE POLICY              |
|NA                                      |
|CERTIFICATE OF SELF-INSURANCE           |
|CERTIFICATE OF DEPOSIT WITH COUNTY JUDGE|
|CERTIFICATE OF DEPOSIT WITH COMPTROLLER |
|SURETY BOND                             |
|PROOF OF LIABILITY INSURANCE            |
+----------------------------------------+



In [ ]:
insuranceCarsDF = joinedDf. \
            filter((
                col("VEH_BODY_STYL_ID").contains("CAR")
                ) & (
                col("FIN_RESP_TYPE_ID") != "NA")
            ).distinct().count()
insuranceCarsDF

11804

In [ ]:
joinedDf. \
            filter((
                col("DAMAGED_PROPERTY").isin(["NO DAMAGE","NONE"])
            ) & ((
                (col("VEH_DMAG_SCL_1_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_1_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            ) | (
                (col("VEH_DMAG_SCL_2_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_2_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            )) & ((
                col("VEH_BODY_STYL_ID").contains("CAR")
                ) & (
                col("FIN_RESP_TYPE_ID") != "NA")
            )).count()

5

In [ ]:
noDamagesDF = joinedDF. \
            filter(col("DAMAGED_PROPERTY").isin(["NO DAMAGE","NONE"]))

damageLevelAbove4DF = joinedDF. \
            filter((
                (col("VEH_DMAG_SCL_1_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_1_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            ) | (
                (col("VEH_DMAG_SCL_2_ID") > "DAMAGED 4") &
                (~col("VEH_DMAG_SCL_2_ID").isin(["NA", "NO DAMAGE", "INVALID VALUE"]))
            ))
        
insuranceCarsDF = joinedDF. \
            filter((
                col("VEH_BODY_STYL_ID").contains("CAR")
                ) & (
                col("FIN_RESP_TYPE_ID") != "NA")
            )
allConditions = insuranceCarsDF.join(damageLevelAbove4DF, on = ['CRASH_ID'], how ='inner').join(noDamagesDF, on = ['CRASH_ID'], how ='inner')
allConditions.select("CRASH_ID").distinct().count()


4

In [ ]:
top_25_state_list = [row[0] for row in Units_df.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).
            groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).limit(25).collect()]
top_10_used_vehicle_colors = [row[0] for row in Units_df.filter(Units_df.VEH_COLOR_ID != "NA").
            groupby("VEH_COLOR_ID").count().orderBy(col("count").desc()).limit(10).collect()]

resultDF = Charges_df.join(Primary_Person_df, on=['CRASH_ID'], how='inner'). \
            join(Units_df, on=['CRASH_ID'], how='inner'). \
            filter(Charges_df.CHARGE.contains("SPEED")). \
            filter(Primary_Person_df.DRVR_LIC_TYPE_ID.isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])). \
            filter(Units_df.VEH_COLOR_ID.isin(top_10_used_vehicle_colors)). \
            filter(Units_df.VEH_LIC_STATE_ID.isin(top_25_state_list)). \
            groupby("VEH_MAKE_ID").count(). \
            orderBy(col("count").desc()).limit(5)
resultDF.show()

+-----------+-----+
|VEH_MAKE_ID|count|
+-----------+-----+
|       FORD|18425|
|  CHEVROLET|16191|
|     TOYOTA|11384|
|      DODGE| 7693|
|     NISSAN| 7047|
+-----------+-----+



# test

In [5]:
!ls

 Volume in drive D has no label.
 Volume Serial Number is 64BC-7D55

 Directory of d:\BCG_Assignment

06-02-2023  17:20    <DIR>          .
06-02-2023  17:20    <DIR>          ..
05-02-2023  14:15                 0 .gitignore
06-02-2023  16:18               800 config.yaml
06-02-2023  17:08    <DIR>          Data
06-02-2023  17:40            61,180 InitialAnalysisPlayground.ipynb
06-02-2023  17:05               536 main.py
06-02-2023  16:40               147 Makefile
06-02-2023  17:04    <DIR>          Output
06-02-2023  17:36             2,562 README.md
06-02-2023  17:39                43 requirements.txt
06-02-2023  17:04    <DIR>          src
               7 File(s)         65,268 bytes
               5 Dir(s)  142,053,920,768 bytes free


In [6]:
%cd ..


[WinError 2] The system cannot find the file specified: 'BCG_Assignment'
d:\BCG_Assignment


In [116]:
!rm -rf *.csv
#!rm -rf BCG_assignment

In [118]:
!unzip BCG_assignment.zip
%cd BCG_assignment

Archive:  BCG_assignment.zip
 extracting: BCG_assignment/.gitignore  
  inflating: BCG_assignment/config.yaml  
   creating: BCG_assignment/Data/
  inflating: BCG_assignment/Data/Charges_use.csv  
  inflating: BCG_assignment/Data/Damages_use.csv  
  inflating: BCG_assignment/Data/Endorse_use.csv  
  inflating: BCG_assignment/Data/Primary_Person_use.csv  
  inflating: BCG_assignment/Data/Restrict_use.csv  
  inflating: BCG_assignment/Data/Units_use.csv  
  inflating: BCG_assignment/main.py  
  inflating: BCG_assignment/Makefile  
  inflating: BCG_assignment/README.md  
   creating: BCG_assignment/src/
   creating: BCG_assignment/src/utilities/
  inflating: BCG_assignment/src/utilities/analyzer.py  
  inflating: BCG_assignment/src/utilities/utils.py  
/content/BCG_assignment


In [119]:
!make build

make: Warning: File 'Makefile' has modification time 19306 s in the future
rm -rf dist
rm -rf Output
mkdir app
cp main.py ./app
cp config.yaml ./app
cp -r Data ./app
cd src && zip -r ../app/src.zip .
  adding: utilities/ (stored 0%)
  adding: utilities/utils.py (deflated 60%)
  adding: utilities/analyzer.py (deflated 76%)
make: warning:  Clock skew detected.  Your build may be incomplete.


In [121]:
!cd app && spark-submit --master "local[*]" --py-files src.zip --files config.yaml main.py && cd ..

23/02/06 11:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/06 11:19:18 INFO SparkContext: Running Spark version 3.3.1
23/02/06 11:19:18 INFO ResourceUtils: ==============================================================
23/02/06 11:19:18 INFO ResourceUtils: No custom resources configured for spark.driver.
23/02/06 11:19:18 INFO ResourceUtils: ==============================================================
23/02/06 11:19:18 INFO SparkContext: Submitted application: USVehicleAccidentAnalysis
23/02/06 11:19:18 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
23/02/06 11:19:18 INFO ResourceProfile: Limiting resource is cpu
23/02/06 11:19:18 INFO ResourcePr

In [ ]:
!rm -rf Output